<a href="https://colab.research.google.com/github/amsac/ML_Notebooks/blob/main/capstone/Labelling_data%26_trial_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring the files and Labelling the data

In [1]:
!pip install transformers
!pip install scikit-learn
!pip install pandas
!pip install torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

# Initialize the BERT tokenizer and model
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [3]:
# Load the log data
log_data = pd.read_csv("OpenStack_2k.log_structured.csv")

# Step 1: Batch Tokenization and Preprocessing
logs = log_data["EventTemplate"].astype(str).tolist()  # Ensure all logs are strings
tokenized = tokenizer(
    logs,
    padding=True,
    truncation=True,
    max_length=100,
    return_tensors="pt"
)

log_tensors = tokenized["input_ids"]
attention_masks = tokenized["attention_mask"]

# Step 2: Compute Log Embeddings
log_embeddings = []
batch_size = 32

for i in range(0, len(log_tensors), batch_size):
    input_ids_batch = log_tensors[i : i + batch_size]
    attn_mask_batch = attention_masks[i : i + batch_size]

    with torch.no_grad():
        outputs = model(input_ids_batch, attention_mask=attn_mask_batch)
    batch_embeddings = outputs.last_hidden_state[:, 0, :].numpy()
    log_embeddings.extend(batch_embeddings)

# Step 3: Anomaly Detection
if len(log_embeddings) > 0:
    mean_embedding = np.mean(log_embeddings, axis=0)
else:
    mean_embedding = None

anomaly_threshold = 0.95
anomalous_indices = []

if mean_embedding is not None:
    for i, log_embedding in enumerate(log_embeddings):
        similarity_score = cosine_similarity([log_embedding], [mean_embedding])[0][0]
        if similarity_score < anomaly_threshold:
            anomalous_indices.append(i)
            print(f"Anomaly detected: Log {i} (Similarity: {similarity_score:.4f})")

# Save results
log_data["Anomaly"] = 0
log_data.loc[anomalous_indices, "Anomaly"] = 1

output_file = "OpenStack_2k.log_labeled.csv"
log_data.to_csv(output_file, index=False)
print(f"Saved labeled data to {output_file}")


Anomaly detected: Log 6 (Similarity: 0.9500)
Anomaly detected: Log 7 (Similarity: 0.9498)
Anomaly detected: Log 8 (Similarity: 0.9100)
Anomaly detected: Log 11 (Similarity: 0.9465)
Anomaly detected: Log 12 (Similarity: 0.9401)
Anomaly detected: Log 20 (Similarity: 0.9394)
Anomaly detected: Log 22 (Similarity: 0.9319)
Anomaly detected: Log 23 (Similarity: 0.9314)
Anomaly detected: Log 24 (Similarity: 0.9396)
Anomaly detected: Log 25 (Similarity: 0.9100)
Anomaly detected: Log 26 (Similarity: 0.9319)
Anomaly detected: Log 27 (Similarity: 0.9465)
Anomaly detected: Log 28 (Similarity: 0.9401)
Anomaly detected: Log 29 (Similarity: 0.9399)
Anomaly detected: Log 33 (Similarity: 0.7489)
Anomaly detected: Log 34 (Similarity: 0.8713)
Anomaly detected: Log 35 (Similarity: 0.9105)
Anomaly detected: Log 36 (Similarity: 0.9349)
Anomaly detected: Log 37 (Similarity: 0.9465)
Anomaly detected: Log 38 (Similarity: 0.9401)
Anomaly detected: Log 46 (Similarity: 0.8966)
Anomaly detected: Log 47 (Similarity:

In [5]:
from google.colab import files
files.download("OpenStack_2k.log_labeled.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Model Exploration

In [ ]:
!pip install git+https://github.com/allegroai/clearml
!pip install clearml-agent



In [ ]:
! export MPLBACKEND=TkAg

In [ ]:
from clearml import Task

Task.set_credentials(
     api_host="https://api.clear.ml",
     web_host="https://app.clear.ml",
     files_host="https://files.clear.ml",
     key='KIY6L46QO9AU1ATGSZOH9R5ZFZ3PXJ',
     secret='xLCc4IboVGjyrfG2H3ia_ck3gInmzf9mj8IZGhLUN0QMYj4ZJ3TgNS2NAPxYrTzMWJc'
)

In [ ]:
!clearml-agent daemon --queue default &

In [ ]:
# create example dataset
from clearml import StorageManager, Dataset

# # Download sample csv file
# csv_file = StorageManager.get_local_copy(
#     remote_url="https://vincentarelbundock.github.io/Rdatasets/csv/AER/Affairs.csv"
# )

# Create a dataset with ClearML`s Dataset class
dataset = Dataset.create(
    dataset_project="GenAI_OpenStack_RCA", dataset_name="OpenStack_2k.log_structured"
)

# add the example csv
dataset.add_files(path='/content/OpenStack_2k.log_structured.csv')

# Upload dataset to ClearML server (customizable)
dataset.upload()

# commit dataset changes
dataset.finalize()


In [ ]:
# Log model to ClearML
from clearml import Logger, Task
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import optimizers

class ModelTrainer:
    def __init__(self, model):
        self.model = model
        self.task = Task.current_task()  # Make sure a ClearML task is initialized

    def prepare_data(self, log_file, batch_size):
        # Dummy implementation – replace with your actual data pipeline
        import tensorflow as tf
        dataset = tf.data.Dataset.from_tensor_slices(([0]*100, [0]*100))
        dataset = dataset.batch(batch_size)
        return dataset

    def train_model(self, log_file, epochs=3, batch_size=16, learning_rate=2e-5):
        # Prepare data
        dataset = self.prepare_data(log_file, batch_size)

        # Compile the model
        optimizer = optimizers.Adam(learning_rate=learning_rate)
        self.model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

        # Create a custom ClearML callback
        class ClearMLCallback(Callback):
            def on_epoch_end(self, epoch, logs=None):
                logs = logs or {}
                Logger.current_logger().report_scalar("loss", "train", iteration=epoch, value=logs.get("loss", 0))
                Logger.current_logger().report_scalar("accuracy", "train", iteration=epoch, value=logs.get("accuracy", 0))

        # Add ClearML callback
        clearml_callback = ClearMLCallback()

        # Train the model
        self.model.fit(dataset, epochs=epochs, callbacks=[clearml_callback])

        # Save and log the model artifact
        model_path = "trained_model.h5"
        self.model.save(model_path)  # Save the model locally
        self.task.upload_artifact("Trained_Model", artifact_object=model_path)

In [ ]:
# add simple test logs/metrics etc in clearml
from clearml import Task
from clearml import Logger
import clearml

def load_model():
    # Define the model architecture
    model = LogAnomalyDetector(base_model='bert-base-uncased')

    # Log the model structure (architecture) as text
    task.get_logger().report_text("Model Architecture:\n" + str(model))

    # Load the state dictionary into the model
    state_dict_path = './logbert_anomaly_model_tf/model.pth'
    model.load_state_dict(
      torch.load(state_dict_path, map_location=torch.device('cpu'), weights_only=True)
    )
    model.eval()

    # Log the model file as an artifact
    task.upload_artifact("LogAnomalyDetector Model State", artifact_object=state_dict_path)

    print("Model loaded and logged in ClearML:", model)
    return model




Train model pipeline

In [ ]:

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, optimizers
from clearml import Logger
import clearml
from tensorflow.keras.callbacks import Callback

class LogAnomalyDetector(tf.keras.Model):
    """
    Anomaly Detection Model based on BERT with TensorFlow/Keras
    """
    def __init__(self, base_model_name='bert-base-uncased', max_length=128, **kwargs): # Added **kwargs to handle extra arguments
        super(LogAnomalyDetector, self).__init__(**kwargs) # Pass extra arguments to the superclass
        self.max_length = max_length
        self.bert = TFBertModel.from_pretrained(base_model_name)
        self.anomaly_head = models.Sequential([
            layers.Dense(128, activation='relu'),
            layers.Dense(1, activation='sigmoid')
        ])

    def call(self, inputs, training=False):
        # Access input_ids and attention_mask using keys instead of unpacking
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        bert_outputs = self.bert(input_ids, attention_mask=attention_mask, training=training)
        cls_embedding = bert_outputs.last_hidden_state[:, 0, :]  # [CLS] token
        return self.anomaly_head(cls_embedding)


class LogAnomalyDetectionPipeline:
    """
    End-to-end pipeline for log anomaly detection using TensorFlow
    """
    def __init__(self, model_name='bert-base-uncased', max_length=128, save_path='./logbert_anomaly_model_tf'):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = LogAnomalyDetector(base_model_name=model_name, max_length=max_length)
        self.save_path = save_path
        self.max_length = max_length

    # def prepare_data(self, log_file, batch_size=16):
    #     # Load log data
    #     log_data = pd.read_csv(log_file)
    #     logs = log_data['EventTemplate'].tolist()

    #     # Tokenize logs
    #     inputs = self.tokenizer(
    #         logs,
    #         max_length=self.max_length,
    #         padding='max_length',
    #         truncation=True,
    #         return_tensors='tf'
    #     )

    #     dataset = tf.data.Dataset.from_tensor_slices((
    #         {'input_ids': inputs['input_ids'], 'attention_mask': inputs['attention_mask']},
    #         tf.zeros(len(logs))  # Dummy labels for unsupervised learning
    #     )).batch(batch_size)
    #     return

    def prepare_data(self, log_file, batch_size=16):
        # Load log data
        log_data = pd.read_csv(log_file)
        logs = log_data['EventTemplate'].tolist()

        # Tokenize logs
        inputs = self.tokenizer(
            logs,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='tf'
        )

        dataset = tf.data.Dataset.from_tensor_slices((
            {'input_ids': inputs['input_ids'], 'attention_mask': inputs['attention_mask']},
            tf.zeros(len(logs))  # Dummy labels for unsupervised learning
        )).batch(batch_size)
        return dataset # Added return statement to return the dataset


    def train_model(self, log_file, epochs=3, batch_size=16, learning_rate=2e-5):
        # Prepare data
        dataset = self.prepare_data(log_file, batch_size)

        # Compile the model
        optimizer = optimizers.Adam(learning_rate=learning_rate)
        self.model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

        # Create a custom ClearML callback
        class ClearMLCallback(Callback):
            def on_epoch_end(self, epoch, logs=None):
                # Log training metrics
                Logger.current_logger().report_scalar("loss", "train", iteration=epoch, value=logs["loss"])
                Logger.current_logger().report_scalar("accuracy", "train", iteration=epoch, value=logs["accuracy"])

        # Add ClearML callback
        clearml_callback = ClearMLCallback()

        # Train the model
        self.model.fit(dataset, epochs=epochs, callbacks=[clearml_callback])

        # Save and log the model artifact
        # model_path = "trained_model.h5"
        model_path = self.model.save("trained_model_new.keras")
        self.model.save(model_path)  # Save the model locally
        task.upload_artifact("Trained_Model", artifact_object=model_path)  # Log model to ClearML


    # def train_model(self, log_file, epochs=3, batch_size=16, learning_rate=2e-5):
    #     # Prepare data
    #     dataset = self.prepare_data(log_file, batch_size)

    #     # Compile the model
    #     optimizer = optimizers.Adam(learning_rate=learning_rate)
    #     self.model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    #     # Train the model
    #     self.model.fit(dataset, epochs=epochs)

    def save_model(self):
        """Save the model and tokenizer"""
        os.makedirs(self.save_path, exist_ok=True)
        self.model.save(f"{self.save_path}/model.h5")
        self.tokenizer.save_pretrained(self.save_path)
        print(f"Model saved to {self.save_path}")

    def load_model(self):
        """Load the model and tokenizer"""
        self.model = tf.keras.models.load_model(f"{self.save_path}/model.h5", custom_objects={'LogAnomalyDetector': LogAnomalyDetector})
        self.tokenizer = BertTokenizer.from_pretrained(self.save_path)
        print("Model loaded successfully")

    def detect_anomaly(self, log_message, threshold=0.5):
        """
        Predict anomaly score for a single log message.
        """
        inputs = self.tokenizer(
            log_message,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='tf'
        )

        # Get prediction
        # anomaly_score = self.model([inputs['input_ids'], inputs['attention_mask']])
        anomaly_score = self.model({
            'input_ids': inputs['input_ids'],
            'attention_mask': inputs['attention_mask']
        })
        score = float(anomaly_score[0][0])

        return {
            'log_message': log_message,
            'anomaly_score': score,
            'is_anomaly': score > threshold
        }


In [ ]:
from clearml import Task

# Initialize ClearML task
task = Task.init(
    project_name="GenAI_OpenStack_RCA",  # Project name
    task_name="train_AnomalyModel",     # Task name
    task_type=Task.TaskTypes.training   # Task type
)

def main():
    # Initialize pipeline
    print("Initialize pipeline")
    pipeline = LogAnomalyDetectionPipeline()

    # Train the model
    print("Train the model")
    pipeline.train_model(
        log_file='/content/OpenStack_2k.log_structured.csv',
        epochs=1
    )

    # # Save the model
    # print("Save the model")
    # pipeline.save_model()

    # # Load the model
    # print("Load the model")
    # pipeline.load_model()
    # pipeline.save_model("anomaly_model.h5")
    # Example anomaly detection
    print("Example anomaly detection")
    sample_log = "nova-compute.log.1.2017-05-16_13:55:31	2017-05-16	00:00:13.658	2931	INFO	nova.compute.resource_tracker	req-addc1839-2ed5-4778-b57e-5854eb7b8b09 - - - - -	Auditing locally available compute resources for node cp-1.slowvm1.tcloud-pg0.utah.cloudlab.us	E28	Auditing locally available compute resources for node <*>"
    anomaly_result = pipeline.detect_anomaly(sample_log)
    print(anomaly_result)

if __name__ == "__main__":
    main()

In [ ]:
class InferencePipeline:
    def __init__(self, model, tokenizer, max_length=128):
        self.model = model
        self.tokenizer = tokenizer
        self.max_length = max_length

    def detect_anomaly(self, log_message, threshold=0.5):
        # Tokenize input log message
        inputs = self.tokenizer(
            log_message,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='tf'
        )

        # Predict using model
        outputs = self.model({
            'input_ids': inputs['input_ids'],
            'attention_mask': inputs['attention_mask']
        }, training=False)

        # Extract scalar anomaly score
        score = float(outputs.numpy().flatten()[0])

        return {
            'log_message': log_message,
            'anomaly_score': score,
            'is_anomaly': score > threshold
        }


In [ ]:
# Load tokenizer and model
from transformers import BertTokenizer
from tensorflow.keras.models import load_model

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Load your saved model (after training)
model = load_model("trained_model_new.keras", custom_objects={'LogAnomalyDetector': LogAnomalyDetector})

# Now create the pipeline
pipeline = InferencePipeline(model=model, tokenizer=tokenizer)

# Test with a sample log
sample_log = "nova-compute.log.1.2017-05-16_13:55:31	2017-05-16	00:00:13.658	2931	INFO	nova.compute.resource_tracker	req-addc1839-2ed5-4778-b57e-5854eb7b8b09 - - - - -	Auditing locally available compute resources for node cp-1.slowvm1.tcloud-pg0.utah.cloudlab.us	E28	Auditing locally available compute resources for node <*>"
result = pipeline.detect_anomaly(sample_log)
print(result)


In [ ]:
from tensorflow.keras.models import load_model
from transformers import BertTokenizer

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Load model
# model = load_model("trained_model_new", custom_objects={'LogAnomalyDetector': LogAnomalyDetector})
model = load_model("trained_model_new.keras", custom_objects={'LogAnomalyDetector': LogAnomalyDetector})


